## 필요한 라이브러리 임포트하기

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

## 데이터 불러오기

In [2]:
# 데이터 불러오기
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# 데이터 확인
train_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
test_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [4]:
train_df.shape, test_df.shape

((20758, 18), (13840, 17))

## 피처 엔지니어링

### BMI 생성

In [5]:
# BMI feature 추가
train_df['BMI'] = train_df['Weight'] / (train_df['Height']**2)
test_df['BMI'] = test_df['Weight'] / (test_df['Height']**2)

# id 컬럼 제거
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

# 성별을 숫자형 데이터로 변환
train_df['Gender'] = train_df['Gender'].map({'Male': 0, 'Female': 1})
test_df['Gender'] = test_df['Gender'].map({'Male': 0, 'Female': 1})

## 데이터 분할

In [6]:
# X와 y 정의
X = train_df.drop('NObeyesdad', axis=1)
y = train_df['NObeyesdad']

# 범주형 컬럼을 One-Hot Encoding으로 변환
X_encoded = pd.get_dummies(X, drop_first=True)
test_df_encoded = pd.get_dummies(test_df, drop_first=True)

In [7]:
# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# X_train에만 있는 컬럼을 test_df_encoded에 추가
for col in X_train.columns:
    if col not in test_df_encoded.columns:
        test_df_encoded[col] = 0

# 순서 일치시키기
test_df_encoded = test_df_encoded[X_train.columns]
        
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_df_scaled = scaler.transform(test_df_encoded)

## 모델 훈련 및 교차 검증

In [8]:
# 랜덤 포레스트 분류기 인스턴스화 및 훈련
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)  

# 교차 검증
cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=5) 
print(f"cv 평균 점수: {cv_scores.mean()}")

cv 평균 점수: 0.899675336976796


## 하이퍼 파라미터 튜닝 - Grid Search

In [ ]:
# Grid Search를 사용한 하이퍼 파라미터 튜닝에 대한 확장된 파라미터 그리드
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리의 개수
    'max_depth': [10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],  # 리프 노드가 되기 위한 최소 샘플 수
    'max_features': ['auto', 'sqrt', 'log2'],  # 최적의 분할을 찾기 위해 고려할 특성의 최대 수
    'criterion': ['gini', 'entropy']  # 불순도를 측정하는 기준
}

# GridSearchCV 인스턴스 생성
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# X_train_scaled, y_train는 사전에 준비된 정규화된 훈련 데이터와 타겟 변수
grid_search.fit(X_train_scaled, y_train)

# 최적의 파라미터와 최고 점수 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 점수:", grid_search.best_score_)

## 최종 모델 평가

In [ ]:
# 테스트 데이터에 대한 예측 및 평가
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

In [ ]:
# 테스트 데이터에 대한 예측 수행
preds_test_df = best_clf.predict(X_test_df_scaled)
print(preds_test_df)

In [ ]:
submission = pd.read_csv("data/sample_submission.csv")

In [ ]:
submission['NObeyesdad'] = preds_test_df
submission.to_csv("submission_dtree.csv", index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import lightgbm as lgb
import time

def convert_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return hours, minutes, seconds

# 데이터셋 준비 (이미 전처리된 데이터셋 가정)
# X_train, X_test, y_train, y_test를 이전 단계에서 준비했다고 가정

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', lgb.LGBMClassifier(random_state=42))
])

# 시간 측정 시작
start_time = time.time()

# 모델 학습
pipeline.fit(X_train, y_train)

# 시간 측정 종료
end_time = time.time()

# 시간 변환
hours, minutes, seconds = convert_seconds_to_hms(end_time - start_time)

# 최종 시간 출력
print(f"학습시간: {hours}시간, {minutes}분, {seconds}초")